## Assignment - 2
### Kaustav Vats (2016048)

In [1]:
import codecs
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re, os
import pickle
from matplotlib import pyplot as plt
from random import randint as randi
from math import log10
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [2]:
def GetAllTokens(fileNameList):
    tokens = []
    for fileName in fileNameList:
        file = open("20_newsgroups/" + foldersName[i] + "/" + fileName, 'r')
        text = pre_processing(file.read())
        tokens += word_tokenize(text)
        stop_words = set(stopwords.words('english')) 
        tokens = [w for w in tokens if not w in stop_words]
        file.close()
    return tokens

def GetTokens(folderName):
    folderPath = "20_newsgroups/" + folderName + "/"
    All_folders = os.listdir(folderPath)
    TestFolderCount = int(len(All_folders) * 0.3)
    Test_Tokens = []
    Train_Tokens = []

    for i in range(TestFolderCount):
        index = randi(0, len(All_folders))
        doc = All_folders.pop(index)
        file = open(folderPath + doc, 'r')
        text = pre_processing(file.read())
        temp = word_tokenize(text)
        Test_Tokens += temp
        file.close()
        
    for doc in All_folders:
        file = open(folderPath + doc, 'r')
        text = pre_processing(file.read())
        temp = word_tokenize(text)
        Train_Tokens += temp
        file.close()
        
    return Train_Tokens, Test_Tokens, 
    
def pre_processing(text):
    header = text.find("\n\n")
    text = text[header:]
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^\w]', ' ', text) # Word Preprocessing
    return text

def BagOfWords(tokens):
    bof = {}
    for t in tokens:
        if (t in bof):
            bof[t] += 1
        else:
            bof[t] = 1
    return bof

def TrainTestSplit(folderName):
    folderPath = "20_newsgroups/" + folderName + "/"
    All_folders = os.listdir(folderPath)
    X1, X2 = train_test_split(All_folders, test_size=0.09)
    return X1, X2

In [3]:
foldersName = os.listdir("20_newsgroups/")
print(foldersName)
DataSplit = []
for i in range(len(foldersName)):
    train, test = TrainTestSplit(foldersName[i])
    train += test
    DataSplit.append([train, test])
    
TokensAll = []
for i in range(len(foldersName)):
    t = GetAllTokens(DataSplit[i][0])
    TokensAll.append(t)
    
# TestTokensAll = []
# for i in range(len(foldersName)):
#     t = GetAllTokens(DataSplit[i][1])
#     TestTokensAll.append(t)
    
BagOfWordsAll = []
for i in range(len(foldersName)):
    t = BagOfWords(TokensAll[i])
    BagOfWordsAll.append(t)
    
Vocab = []
for i in range(len(foldersName)):
    Vocab += list(BagOfWordsAll[i].keys())
Vocab = list(set(Vocab))
VocabDic = {}
for i in range(len(Vocab)):
    VocabDic[Vocab[i]] = i
print(len(Vocab))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
168574


In [4]:
def Fit(K=1):
    LiklihoodProba = np.zeros((len(Vocab), len(foldersName)))
    if (os.path.isfile("Likli_{}".format(K))):
        LiklihoodProba = np.load("Likli_{}".format(K))
        return LiklihoodProba
    for i in tqdm(range(len(Vocab))):
        word = Vocab[i]
        for j in range(len(foldersName)):
            bow = BagOfWordsAll[j]
            if (word in bow):
                wc = bow[word] + K
                den = len(TokensAll[j]) + K*len(Vocab)
                LiklihoodProba[i, j] = wc/den
            else:
                wc = K
                den = len(TokensAll[j]) + K*len(Vocab)
                LiklihoodProba[i, j] = wc/den
    np.save("Likli_{}".format(K), LiklihoodProba)
    return LiklihoodProba
TotalFiles = 0
for i in range(20):
    TotalFiles += len(DataSplit[i][0])
Priors = []
for i in range(20):
    Priors.append(len(DataSplit[i][0])/TotalFiles)
print(Priors)

[0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.049857478621793266, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774, 0.050007501125168774]


In [5]:
for k in [1, 5, 10, 100]:
    Fit(K=k)

In [6]:
def Predict(tokens, LiklihoodProba, K=1):
    probs = np.zeros(len(foldersName))
    for i in range(len(Priors)):
        probs[i] = log10(Priors[i])
    for j in range(len(foldersName)):
        for i in range(len(tokens)):
            if (tokens[i] != -1):
                probs[j] += log10(LiklihoodProba[tokens[i], j])
            else:
                probs[j] += log10(K/(len(TokensAll[j]) + K*len(Vocab)))
    return np.argmax(probs)


In [7]:
DocPerClass = []
for i in range(20):
    ClassDoc = []
    tokens = []
    for j in tqdm(range(len(DataSplit[i][1]))):
        file = open("20_newsgroups/" + foldersName[i] + "/" + DataSplit[i][1][j], 'r')
        text = pre_processing(file.read())
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english')) 
        tokens = [w for w in tokens if not w in stop_words]
        Final = []
        for w in tokens:
            if w in Vocab:
                Final.append(VocabDic[w])
            else:
                Final.append(-1)
        file.close()
        ClassDoc.append(Final)
    DocPerClass.append(ClassDoc)

KeyboardInterrupt: 

In [21]:
for k in [1, 5, 10, 100]:
    LiklihoodProba = Fit(K=k)
    print("[+] Training Complete")
    correct = 0
    for i in tqdm(range(20)):
        for j in range(len(DocPerClass[i])):
            c = Predict(DocPerClass[i][j], LiklihoodProba, K=k)
            if (c == i):
                correct += 1
    
    Denom = 0
    for i in range(20):
        Denom += len(DataSplit[i][1])
    print("Accuracy[K={}]: ".format(k), correct/Denom)

[+] Training Complete


Accuracy[K=1]:  0.8216666666666667


[+] Training Complete


Accuracy[K=5]:  0.7805555555555556


[+] Training Complete


Accuracy[K=10]:  0.7394444444444445


[+] Training Complete


Accuracy[K=100]:  0.5472222222222223
